In [18]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import sdss
import pandas as pd
from astropy.io import fits
from astropy import units as u

In [19]:
#objets autour des cooredonnées indiquées 

#'01h26m05.63s +4d54m23.3s' --> 6 redshifts 
coordonnees = '01h26m05.63s +4d54m23.3s'


co = coords.SkyCoord(coordonnees)
result = SDSS.query_region(co, radius = '3 arcmin')
print(len(result))

192


In [20]:
#récupération de leur ID 
inter = result[:351]
tab = inter["objid"]
tab_str = [str(objid) for objid in tab]
objid_list = ', '.join(tab_str)

In [21]:
#récupération de leurs infos via sql query
sql = f"SELECT ObjID,ra, dec, type,u,g,r,i,z,SpecObjID FROM PhotoObj WHERE objID IN ({objid_list})"
df = sdss.sql2df(sql)

In [22]:
df.to_csv('obj_database.csv')

In [23]:
import pandas as pd

id_rs = {}

df = pd.read_csv('obj_database.csv')

for spec_obj_id in df['SpecObjID']:
    if spec_obj_id != 0 :
        sp = sdss.SpecObj(spec_obj_id)
        sp.download()
        rs = sp.z
        id_rs[spec_obj_id] = rs

df['redshift'] = pd.NA

for index, row in df.iterrows():
    spec_obj_id = row['SpecObjID']
    if pd.notna(spec_obj_id): 
        redshift = id_rs.get(spec_obj_id, pd.NA)
        df.at[index, 'redshift'] = redshift

df.to_csv('obj_database.csv', index=False)

print("Done!!")

Done!!
